In [ ]:
import numpy as np
import scipy
from scipy.optimize import minimize
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [ ]:
class IdentityQuantizer:
    def quantize(xs, rng):
        return xs
    
    @staticmethod
    def homogeneous_ab(n, d):
        return 0, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 * d

class CorrelatedQuantizer:
    def quantize(xs, rng):
        n, d = xs.shape
        r = np.linalg.norm(xs, axis=1).max() # 32 bits per client for norm
        
        xs_normalized = (xs + r) / (2 * r)
        
        gammas = rng.random((n, d)) / n # generate gammas in [0, 1/n)
        perms = rng.permuted(np.tile(np.linspace(0, 1, n, endpoint=False), d).reshape(d, n).T, axis=0)
        thresholds = perms + gammas
        
        xs_compressed = np.zeros_like(xs)
        xs_compressed[xs_normalized > thresholds] = 1 # 1 bit per coordinate per client
        
        xs_decompressed = 2 * r * xs_compressed - r
        return xs_decompressed
    
    @staticmethod
    def homogeneous_ab(n, d):
        return d / 4 / n**2, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 + d


class UncorrelatedQuantizer:
    @staticmethod
    def quantize(xs, rng):
        n, d = xs.shape
        r = np.linalg.norm(xs, axis=1)[:, np.newaxis]
        
        xs_normalized = (xs + r) / (2 * r)
        
        thresholds = rng.random((n, d))
        
        xs_compressed = np.zeros_like(xs)
        xs_compressed[xs_normalized > thresholds] = 1
        
        xs_decompressed = 2 * r * xs_compressed - r
        return xs_decompressed
    
    @staticmethod
    def homogeneous_ab(n, d):
        return d / 4 / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 32 + d


class RandK:
    @staticmethod
    def quantize(xs, rng):
        n, d = xs.shape
    
        # k = int(0.1 * d)
        k = 1
        # Sample a random subset of coordinates without replacement
        idx = rng.random(xs.shape).argsort(1)[:,:k]
        
        xs_decompressed = np.zeros_like(xs)
        np.put_along_axis(xs_decompressed, idx, np.take_along_axis(xs, idx, axis=1), axis=1)
        return xs_decompressed * (d / k)
    
    @staticmethod
    def homogeneous_ab(n, d):
        # k = int(0.1 * d)
        k = 1
        return (d / k - 1) / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        # k = int(0.1 * d)
        k = 1
        return 32 * k


class PermK:
    def quantize(xs, rng):
        n, d = xs.shape
        if n % d == 0:
            q = n // d
            
            idx = rng.permuted(np.arange(n)).reshape(d, q)
            
            xs_decompressed_t = np.zeros_like(xs).T
            np.put_along_axis(xs_decompressed_t, idx, np.take_along_axis(xs.T, idx, axis=1), axis=1)
            return xs_decompressed_t.T * d
        elif d % n ==0:
            q = d // n
            idx = rng.permuted(np.arange(d)).reshape(n, q)
            
            xs_decompressed = np.zeros_like(xs)
            np.put_along_axis(xs_decompressed, idx, np.take_along_axis(xs, idx, axis=1), axis=1)
            return xs_decompressed * n
        else:
            raise ValueError("n must be a multiple of d or vice versa")
    
    @staticmethod
    def homogeneous_ab(n, d):
        if d > n:
            return 1, 1
        else:
            return 1 - (n - d) / (n - 1), 1 - (n - d) / (n - 1)
        
    @staticmethod
    def bits_per_client(n, d):
        if d > n:
            return 32 * d/n
        else:
            return 32


class Natural:
    def quantize(xs, rng: np.random.Generator):
        n, d = xs.shape

        zeros_pos = (xs == 0)
        xs[zeros_pos] = 1
        
        alpha = np.log2(abs(xs))
        abs_down = np.power(2, np.floor(alpha))
        abs_up = np.power(2, np.ceil(alpha))
        pt = (abs_up - abs(xs)) / abs_down
        sample = rng.binomial(1, pt).astype(bool)
        
        out = np.zeros_like(xs)
        np.putmask(out, sample, abs_down)
        np.putmask(out, ~sample, abs_up)
        np.putmask(out, zeros_pos, 0)
        out *= np.sign(xs)
        return out
    
    @staticmethod
    def homogeneous_ab(n, d):
        return 1 / 8 / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        return 9 * d
    
    
    
def hadamard_transform_(vec):
    """fast Walsh–Hadamard transform (in-place)

    :param vec: vec is expected to be a power of 2!
    :return: the Hadamard transform of vec
    """
    n = vec.shape[0]
    d = vec.shape[1]
    h = 2
    while h <= d:
        hf = h // 2
        vec = vec.reshape(n, d // h, h)

        ## the following is a more inplace way of doing the following:
        # half_1 = batch[:, :, :hf]
        # half_2 = batch[:, :, hf:]
        # batch = torch.cat((half_1 + half_2, half_1 - half_2), dim=-1)
        # the NOT inplace seems to be actually be slightly faster
        # (I assume for making more memory-contiguous operations. That being said,
        # it more easily throws out-of-memory and may slow things overall,
        # so using inplace version below:)

        vec[:, :, :hf] = vec[:, :, :hf] + vec[:, :, hf : 2 * hf]
        vec[:, :, hf : 2 * hf] = vec[:, :, :hf] - 2 * vec[:, :, hf : 2 * hf]
        h *= 2

    vec *= d**-0.5  # vec /= np.sqrt(d)

    return vec.reshape(n, d)


class DriveQuantizer:
    def quantize(xs, rng: np.random.Generator):
        n, d = xs.shape
        
        # Pad
        next_power_of_2 = 1 << (d - 1).bit_length()
        xs = np.pad(xs, ((0, 0), (0, next_power_of_2 - d)))
        
        # Rotate

        rademacher = 2 * rng.binomial(1, 0.5, size=xs.shape).astype(xs.dtype) - 1
        xs = hadamard_transform_(xs * rademacher)

        # Quantize
        normalized_data = xs * np.sqrt(d) / np.linalg.norm(xs, axis=1)[:, np.newaxis]

        quantized_xs = np.zeros_like(xs)
        quantized_xs[normalized_data > 0] = np.sqrt(2/np.pi)
        quantized_xs[normalized_data < 0] = -np.sqrt(2/np.pi)

        # Dequantize
        dequantized_xs = (np.linalg.norm(xs, axis=1) ** 2 / np.sum(xs * quantized_xs, axis=1))[:, np.newaxis] * quantized_xs
        np.nan_to_num(dequantized_xs, copy=False)
        
        # Unrotate
        out = hadamard_transform_(dequantized_xs) * rademacher
        
        # Unpad
        out = out[:,:d]
        return out
    
    @staticmethod
    def homogeneous_ab(n, d):
        return (np.pi / 2 - 1) / n, 0
    
    @staticmethod
    def bits_per_client(n, d):
        next_power_of_2 = 1 << (d - 1).bit_length()
        return 32 + next_power_of_2
    

In [ ]:
class TridiagonalFunctions:
    def __init__(self, tridiag_matrices, biases):
        self.tridiag_matrices = tridiag_matrices
        self.biases = biases
        
        self.mean_matrix = sum(tridiag_matrices) / len(tridiag_matrices)
        self.mean_bias = sum(biases) / len(biases)
        
        self.x = np.zeros((len(self.biases[0]),))
        self.x[0] = np.sqrt(len(self.biases[0]))
        
    def reset(self):
        self.x = np.zeros((len(self.biases[0]),))
        self.x[0] = np.sqrt(len(self.biases[0]))
        
    def get_nd(self):
        return len(self.tridiag_matrices), len(self.biases[0])
        
    def get_grad(self, batch_idx):
        grads = []
        for id in batch_idx:
            tridiag_matrix = self.tridiag_matrices[id]
            bias = self.biases[id]
            grad = tridiag_matrix.dot(self.x) - bias
            grads.append(grad)
        return np.stack(grads)
    
    def get_full_grad(self):
        return self.mean_matrix.dot(self.x) - self.mean_bias
    
    def get_values(self, batch_idx):
        values = []
        for id in batch_idx:
            tridiag_matrix = self.tridiag_matrices[id]
            bias = self.biases[id]
            value = (1/2) * np.dot(self.x, tridiag_matrix.dot(self.x)) - np.dot(bias, self.x)
            values.append(value)
        return np.stack(values)
    
    def get_grad_and_values(self, batch_idx):
        return self.get_grad(batch_idx), self.get_values(batch_idx)
    
    def apply_update(self, update):
        self.x += update
    
    def get_liptschitz_constants(self):
        eighs = [scipy.linalg.eigh_tridiagonal(
            matrix.diagonal(0),
            matrix.diagonal(1),
            eigvals_only=True,
        ) for matrix in self.tridiag_matrices]
        return np.max(np.abs(eighs), axis=1)
    
    def get_smoothness_variance(self):
        weights = np.ones(len(self.tridiag_matrices), dtype=np.float32) / len(self.tridiag_matrices)

        lambda_square_matrix = lambda A: A.dot(A)
        def lambda_mean_matrices(A_list, weights=None):
            if weights is not None:
                A_list_weights = [A / (len(A_list) * weight) for A, weight in zip(A_list, weights)]
            else:
                A_list_weights = A_list
            return sum(A_list_weights) / len(A_list)
        
        mean_matrices = lambda_mean_matrices(self.tridiag_matrices)
        square_mean_matrices = lambda_square_matrix(mean_matrices)
        square_matrices = list(map(lambda_square_matrix, self.tridiag_matrices))
        mean_square_matrices = lambda_mean_matrices(square_matrices, weights)

        result_matrix = mean_square_matrices - square_mean_matrices

        svdvals = scipy.sparse.linalg.svds(result_matrix, k=1, return_singular_vectors=False)
        op_norm = np.max(svdvals)
        if op_norm < 1e-5: # float errors add up
            return 0.0
        svb = np.sqrt(op_norm)
        return svb


def create_worst_case(dim, liptschitz_gradient_constant, noise_lambda, rng, strategy='mul'):
    scale = liptschitz_gradient_constant / 4.
    main_diag = 2 * np.ones((dim,), dtype=np.float32)
    side_diag = -1 * np.ones((dim - 1,), dtype=np.float32)
    b = np.zeros((dim,), dtype=np.float32)
    b[0] = -1
    if noise_lambda > 0:
        if strategy == 'add':
            noise = (noise_lambda * rng.standard_exponential(size=(1,), dtype=np.float32))[0]
            main_diag += noise
            side_diag += noise
        if strategy == 'mul':
            noise_scale = (1 + noise_lambda * rng.standard_normal(size=(1,), dtype=np.float32))[0]
            noise_bias = (noise_lambda * rng.standard_normal(size=(1,), dtype=np.float32))[0]
            b[0] += noise_bias
            b[0] *= noise_scale
            main_diag *= noise_scale
            side_diag *= noise_scale
    b[0] *= scale
    return scale * main_diag, scale * side_diag, b


def create_worst_case_functions(num_nodes, dim, rng, liptschitz_gradient_constant=1, noise_lambda=0, strongly_convex_constant=0.001):
    main_diags = []
    side_diags = []
    bs = []
    for i in range(num_nodes):
        main_diag, side_diag, b = create_worst_case(dim, liptschitz_gradient_constant, noise_lambda, rng)
        main_diags.append(main_diag)
        side_diags.append(side_diag)
        bs.append(b)
    lambda_mean_matrices = lambda A_list: sum(A_list) / len(A_list)
    mean_main_diag = lambda_mean_matrices(main_diags)
    mean_side_diag = lambda_mean_matrices(side_diags)
    eigs = scipy.linalg.eigh_tridiagonal(mean_main_diag, mean_side_diag, eigvals_only=True)
    min_eig = np.min(eigs)
    tridiag_matrices = []
    biases = []
    for main_diag, side_diag, b in zip(main_diags, side_diags, bs):
        main_diag = main_diag - min_eig
        main_diag = main_diag + strongly_convex_constant
        
        tridiag_matrices.append(scipy.sparse.diags([side_diag, main_diag, side_diag], [-1, 0, 1]))
        biases.append(b)
    return TridiagonalFunctions(tridiag_matrices, biases)


def create_convex_different_liptschitz(num_nodes, dim, rng, liptschitz_gradient_constant=1, noise_lambda=0):
    tridiag_matrices = []
    biases = []
    for _ in range(num_nodes):
        main_diag, side_diag, b = create_worst_case(dim, liptschitz_gradient_constant, 0.0, rng)
        scale = 1 + noise_lambda * rng.standard_exponential(size=(1,), dtype=np.float32)
        main_diag *= scale
        side_diag *= scale
        b[0] += noise_lambda * rng.standard_normal(size=(1,), dtype=np.float32)
        tridiag_matrices.append(scipy.sparse.diags([side_diag, main_diag, side_diag], [-1, 0, 1]))
        biases.append(b)
    return TridiagonalFunctions(tridiag_matrices, biases)


def create_convex_max_diff_liptschitz(num_nodes, dim, rng, liptschitz_gradient_constant=1, noise_lambda=0):
    tridiag_matrices = []
    biases = []
    for _ in range(num_nodes - 1):
        main_diag, side_diag, b = create_worst_case(dim, liptschitz_gradient_constant, 1.0, rng)
        b[0] += rng.standard_normal(size=(1,), dtype=np.float32)
        tridiag_matrices.append(scipy.sparse.diags([side_diag, main_diag, side_diag], [-1, 0, 1]))
        biases.append(b)
    
    main_diag, side_diag, b = create_worst_case(dim, liptschitz_gradient_constant, 1.0 + noise_lambda, rng)
    b[0] += rng.standard_normal(size=(1,), dtype=np.float32)
    tridiag_matrices.append(scipy.sparse.diags([side_diag, main_diag, side_diag], [-1, 0, 1]))
    biases.append(b)  
    
    return TridiagonalFunctions(tridiag_matrices, biases)

In [ ]:
def marina_client_step(i: int, broadcasted_grad, functions, p: float, gamma: float, quantize_fn, rng, batch_idx, n):
    c = i == 0 or rng.random() < p
    if c:
        functions.apply_update(- gamma * broadcasted_grad) 
        grads, values = functions.get_grad_and_values(np.arange(0, n))
        uncompressed_updates = grads
        compressed_updates = grads
    else:
        prev_grads, _ = functions.get_grad_and_values(batch_idx)
        functions.apply_update(- gamma * broadcasted_grad) 
        grads, values = functions.get_grad_and_values(batch_idx)
        uncompressed_updates = grads - prev_grads
        compressed_updates = quantize_fn(uncompressed_updates, rng)
    return grads, values, c, uncompressed_updates, compressed_updates


def marina_master_step(updates, prev_grad, c, rng, batch_size, probs, batch_idx, n):
    if c:
        prev_grad = updates.mean(axis=0)
    else:
        prev_grad = prev_grad + updates.mean(axis=0)
    return prev_grad, updates.shape[0]


def weighted_marina_master_step(updates, prev_grad, c, rng, batch_size, probs, batch_idx, n):
    if c:
        prev_grad = updates.mean(axis=0)
        num_unique_clients = n
    else:
        num_unique_clients = len(np.unique(batch_idx))
        sampled_client_updates = updates / probs[batch_idx] / n
        prev_grad = prev_grad + sampled_client_updates.mean(axis=0)
    return prev_grad, num_unique_clients

def marina_homogeneous_p(a, b, d, bits_per_client, liptschitz_constants):
    assert b == 0, "Not implemented"
    if a == 0:
        return 0

    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / len(liptschitz_constants)
    mean_square_liptschitz_gradient_constant = (sum(l**2 for l in liptschitz_constants) / len(liptschitz_constants)) ** (1 / 2)
    a *= (mean_square_liptschitz_gradient_constant / mean_liptschitz_gradient_constant) ** 2
    
    c = bits_per_client / (32 * d)
    
    def func_(p):
        return (p + (1 - p) * c) * (1 + np.sqrt((1 - p) / p * a))

    def grad_(p):
        return ((-a / p - a * (1 - p) / p**2) * (p + c * (1 - p))) / (2 * np.sqrt(a * (1 - p) / p)) + (1 - c) * (
            1 + np.sqrt(a * (1 - p) / p)
        )

    result = minimize(func_, 0.5, method="SLSQP", jac=grad_, bounds=[(1e-5, 1 - 1e-5)], tol=1e-10)
    assert result.success, result.message
    return result.x[0]


def marina_homogeneous_gamma(a, b, liptschitz_constants, smoothness_variance, p, num_clients):
    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / num_clients
    mean_square_liptschitz_gradient_constant = (sum(l**2 for l in liptschitz_constants) / num_clients) ** (1 / 2)
    
    if b == 0:
        smoothness_variance = 0 # Doesn't matter
    if a == 0:
        p = 1 # Doesn't matter

    m = mean_liptschitz_gradient_constant + np.sqrt(
        ((1 - p) / p) * ((a - b) * mean_square_liptschitz_gradient_constant**2 + b * smoothness_variance**2)
    )
    
    return 1/m


def weighted_marina_homogeneous_p(a, b, d, bits_per_client, n):
    assert b == 0, "Not implemented"
    
    a = a * n + 1
    c = bits_per_client / (32 * d * n)
    
    def func_(p):
        return (p + (1 - p) * c) * (1 + np.sqrt((1 - p) / p * a))

    def grad_(p):
        return ((-a / p - a * (1 - p) / p**2) * (p + c * (1 - p))) / (2 * np.sqrt(a * (1 - p) / p)) + (1 - c) * (
            1 + np.sqrt(a * (1 - p) / p)
        )

    result = minimize(func_, 0.5, method="SLSQP", jac=grad_, bounds=[(1e-5, 1 - 1e-5)], tol=1e-10)
    assert result.success, result.message
    return result.x[0]


def weighted_marina_homogeneous_gamma(a, b, liptschitz_constants, p, num_clients):
    assert b == 0
    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / num_clients
    w = a * num_clients
    return 1 / mean_liptschitz_gradient_constant / (1 + np.sqrt((1 - p) / p * (1 + w)))

In [ ]:
def dcgd_client_step(i: int, broadcasted_grad, functions, p: float, gamma: float, quantize_fn, rng, batch_idx, n):
    prev_grads, _ = functions.get_grad_and_values(batch_idx)
    functions.apply_update(- gamma * broadcasted_grad) 
    grads, values = functions.get_grad_and_values(batch_idx)
    uncompressed_updates = grads
    compressed_updates = quantize_fn(uncompressed_updates, rng)
    return grads, values, False, uncompressed_updates, compressed_updates


def dcgd_master_step(updates, prev_grad, c, rng, batch_size, probs, batch_idx, n):
    prev_grad = updates.mean(axis=0)
    return prev_grad, updates.shape[0]


def dcgd_homogeneous_gamma(a, b, liptschitz_constants, smoothness_variance, p, num_clients):
    assert b == 0
    mean_liptschitz_gradient_constant = sum(liptschitz_constants) / num_clients
    w = a * num_clients
    return 1 / mean_liptschitz_gradient_constant / (1 + w)


In [ ]:
def measure(functions, quantizer, gamma_mult, rng, p=None, steps=None, comm_limit=None, use_dcgd=False, use_importance_sampling=False, batch_size=None):
    assert (steps is None) ^ (comm_limit is None), "Either steps or comm_limit must be specified"
    if use_importance_sampling:
        assert batch_size is not None, "Batch size must be specified for importance sampling"
        assert not use_dcgd
    
    n, d = functions.get_nd()
    
    a, b = quantizer.homogeneous_ab(n, d)
    liptschitz_constants = np.array(functions.get_liptschitz_constants())
    
    if p is None:
        if use_importance_sampling:
            p = weighted_marina_homogeneous_p(a, b, d, quantizer.bits_per_client(n, d), n)
        elif use_dcgd:
            p = None
        else:
            p = marina_homogeneous_p(a, b, d, quantizer.bits_per_client(n, d), liptschitz_constants)
    
    if use_importance_sampling:
        gamma_base = weighted_marina_homogeneous_gamma(a, b, liptschitz_constants, p, n)
    elif use_dcgd:
        gamma_base = dcgd_homogeneous_gamma(a, b, liptschitz_constants, functions.get_smoothness_variance(), p, n)
    else:
        gamma_base = marina_homogeneous_gamma(a, b, liptschitz_constants, functions.get_smoothness_variance(), p, n)
    
    gamma = gamma_mult * gamma_base
    
    # print(f"{p=}, {gamma=}")

    metrics = {
        'values': [],
        'grad_norms_2': [],
        'total_bits_received': [],
        'is_full': [],
        'r': [],
        'sigma': [],
        'updates_mean_squares': [],
        'sigma_r_norm': [],
        'step_size': gamma,
        'p': p,
    }
    
    master_prev_grad = np.zeros((d,))
    
    total_bits_received = 0
    broadcasted_grad = np.zeros((d, ))
    if steps is not None:
        pbar = tqdm(total=steps)
    else:
        pbar = tqdm(total=comm_limit)
        
        
    if use_importance_sampling:
        client_step_fn = marina_client_step
        master_step_fn = weighted_marina_master_step
    elif use_dcgd:
        client_step_fn = dcgd_client_step
        master_step_fn = dcgd_master_step
    else:
        client_step_fn = marina_client_step
        master_step_fn = marina_master_step
        
        
    step = 0
    while True:
        if use_importance_sampling:
            batch_idx = np.random.choice(n, p=liptschitz_constants / liptschitz_constants.sum(), size=batch_size)
        else:
            batch_idx = np.arange(0, n)
        
        grads, values, c, uncompressed_updates, compressed_updates = client_step_fn(
            step, 
            broadcasted_grad, 
            functions,
            p, 
            gamma, 
            quantizer.quantize,
            rng,
            batch_idx=batch_idx,
            n=n,
        )
        
        broadcasted_grad, num_unique_clients = master_step_fn(
            compressed_updates, 
            master_prev_grad, 
            c, 
            rng, 
            batch_size,
            liptschitz_constants / liptschitz_constants.sum(),
            batch_idx,
            n,
        )

        master_prev_grad = broadcasted_grad
        
        # Metrics
        metrics['values'].append(values.mean())
        metrics['grad_norms_2'].append(np.linalg.norm(functions.get_full_grad())**2)
        metrics['total_bits_received'].append(total_bits_received)
        metrics['is_full'].append(c)
        # metrics['r'].append(np.linalg.norm(uncompressed_updates, axis=1).max())
        # metrics['sigma'].append(np.linalg.norm(uncompressed_updates - uncompressed_updates.mean(axis=0), axis=1).mean())
        # metrics['sigma_r_norm'].append(metrics['sigma'][-1] *  metrics['r'][-1] / (np.linalg.norm(uncompressed_updates, axis=1)**2).mean())
        
        bits_per_client = 32 * d if c else quantizer.bits_per_client(n, d)
        total_bits_received += bits_per_client * num_unique_clients
        step += 1
        
        if steps is not None:
            pbar.update(1)
        else:
            pbar.update(bits_per_client * num_unique_clients)
        
        if steps is not None and step >= steps:
            break
        
        if comm_limit is not None and total_bits_received >= comm_limit:
            break
        
    return metrics

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines


LINE_KWARGS = {
    # Baseline
    "DCGD + IQ": {"color": "black"},
    "DCGD + CQ": {"color": "gray", "linestyle": (0, (5, 1))},
    "Gradient Descent": {"color": "gray", "linestyle": "solid"},
    "MARINA + CQ": {"color": "green", "linestyle": "dashed"}, 
    "MARINA + IQ":{"color": "red", "linestyle": "dashdot"},
    "MARINA + DRIVE": {"color": "blue", "linestyle": "dotted"},
    # Weights
     "MARINA + DRIVE + IS": {"color": "midnightblue", "linestyle": (0, (1, 1))}, 
}
DOT_KWARGS = {
    # Baseline
    "DCGD + IQ": {"color": "black"},
    "DCGD + CQ": {"color": "gray"},
    "MARINA + CQ": {"color": "green", "marker": "^"}, 
    "MARINA + IQ":{"color": "red", "linestyle": ":", "marker": "v"}, 
    "MARINA + DRIVE": {"color": "blue", "linestyle": "--", "marker": "o"},
    # Weights
    "MARINA + DRIVE + IS": {"color": "midnightblue", "marker": "*"}, 
}


def plot_norms(ax, metrics, y_key, label, line_kwargs, dot_kwargs, weighted):
    full_grad = np.array(metrics["is_full"])
    bits_per_client = np.array(metrics["total_bits_received"]) / NUM_CLIENTS
    grad_norm_2 = np.array(metrics[y_key])
    
    c = ~np.concatenate((full_grad, [True])) & ~np.concatenate(([True], full_grad))
    
    diff = np.diff(c.astype(int))
    starts = np.where(diff == 1)[0]
    ends = np.where(diff == -1)[0]
    
    for start, end in zip(starts, ends):
        norms = np.minimum(grad_norm_2[start:end+1], 10**20) # for debugging purposes 
        ax.plot(bits_per_client[start:end+1], norms, label=label, **line_kwargs)
    
    # Connect points inbetween full grads
    for i in range(len(bits_per_client)-1):
        if not full_grad[i] and not full_grad[i+1]:
            pass
        elif full_grad[i+1]:
            norm = min(grad_norm_2[i], 10**20) # for debugging purposes 
            ax.scatter(
                bits_per_client[i],
                norm,
                **dot_kwargs,
            )
            
            
def plot(weighted: bool):
    num_plots = len(NOISES)
    if weighted:
        fig, axs = plt.subplots(2, 1, figsize=(6, 4))
    else:
        fig, axs = plt.subplots(num_plots, 1, figsize=(6, 6))

    for i, noise in enumerate(NOISES):
        ax = axs[i]
        
        for name, metrics in METRICS.items():
            plot_norms(
                ax, 
                metrics[noise], 
                "grad_norms_2", 
                f"{name}" + (f"; {GAMMAS[name][noise]}x Step Size" if name != "MARINA + CQ" or noise == 0 else ""), 
                LINE_KWARGS[name], 
                DOT_KWARGS[name],
                weighted
            )

        ax.set_yscale("log")

        ax.set_xlabel("# of bits sent per client")
        ax.set_ylabel(r"$\|\nabla f(x)\|^2$")
        
        if weighted:
            ax.set_title(rf"$\min L_i=$ {PROBLEM_PROPERTIES[noise]['minLi']:.2}; $\max L_i=$ {PROBLEM_PROPERTIES[noise]['maxLi']:.2}")
        else:
            ax.set_title(rf"$L_\pm=$ {PROBLEM_PROPERTIES[noise]['L+-']:.2}; $L_-=$ {PROBLEM_PROPERTIES[noise]['L-']:.2}; $L_+=$ {PROBLEM_PROPERTIES[noise]['L+']:.2}")
        
        # Legend
        handles = []
        for name in METRICS:
            handle_kwargs = {}
            handle_kwargs.update(DOT_KWARGS[name])
            handle_kwargs.update(LINE_KWARGS[name])
            handle = mlines.Line2D([], [], 
                label=f"{name}" + (f"; {GAMMAS[name][noise]}x Step Size" if name != "MARINA + CQ" or noise == 0 else ""), 
                **handle_kwargs
            )
            handles.append(handle)
        ax.legend(handles=handles, loc='upper right')
    
    fig.tight_layout()
    return fig

## 4.2 Quadratic Optimization Tasks with Various Hessian Variances $L_\pm$

In [ ]:
SIZE = 1024
NUM_CLIENTS = 128

SEED = 0
RNG = np.random.default_rng(SEED)

In [ ]:
DCGD_UNCORR_GAMMA_MULT = {0.0: 64, 0.1: 64, 0.5: 64, 1.0: 32}
DCGD_CORR_GAMMA_MULT = {0.0: 2, 0.1: 2, 0.5: 2, 1.0: 2}
CORR_GAMMA_MULT = {0.0: 4, 0.1: 4, 0.5: 2, 1.0: 2}
UNCORR_GAMMA_MULT = {0.0: 1, 0.1: 1, 0.5: 1, 1.0: 1}
DRIVE_GAMMA_MULT = {0.0: 2, 0.1: 2, 0.5: 2, 1.0: 2}
NOISES = [0.1, 0.5, 1.0]

START = np.zeros((SIZE,))
START[0] = np.sqrt(SIZE)

DCGD_UNCORR_METRICS = {}
DCGD_CORR_METRICS = {}
CORR_METRICS = {}
UNCORR_METRICS = {}
DRIVE_METRICS = {}

PROBLEM_PROPERTIES = {noise: {} for noise in NOISES}

In [ ]:
COMM_LIMIT = 3e8

for i, noise in enumerate(NOISES):
    print(f"noise = {noise}")
    functions = create_worst_case_functions(NUM_CLIENTS, SIZE, RNG, noise_lambda=noise)
    
    PROBLEM_PROPERTIES[noise]["L+-"] = functions.get_smoothness_variance()
    liptschitz_constants = functions.get_liptschitz_constants()
    PROBLEM_PROPERTIES[noise]["L-"] = sum(liptschitz_constants) / NUM_CLIENTS
    PROBLEM_PROPERTIES[noise]["L+"] = (sum(l**2 for l in liptschitz_constants) / NUM_CLIENTS) ** (1 / 2)
    
    functions.reset()
    DCGD_UNCORR_METRICS[noise] = measure(
        functions, UncorrelatedQuantizer, DCGD_UNCORR_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT, use_dcgd=True,
    )
    # functions.reset()
    # DCGD_CORR_METRICS[noise] = measure(
    #     functions, CorrelatedQuantizer, DCGD_CORR_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT, use_dcgd=True,
    # )
    functions.reset()
    CORR_METRICS[noise] = measure(
        functions, CorrelatedQuantizer, CORR_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT,
    )
    functions.reset()
    UNCORR_METRICS[noise] = measure(
        functions, UncorrelatedQuantizer, UNCORR_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT,
    )
    functions.reset()
    DRIVE_METRICS[noise] = measure(
        functions, DriveQuantizer, DRIVE_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT,
    )


In [ ]:
METRICS = {
    "DCGD + IQ": DCGD_UNCORR_METRICS,
    # "DCGD + CQ": DCGD_CORR_METRICS,
    "MARINA + CQ": CORR_METRICS,
    "MARINA + IQ": UNCORR_METRICS, 
    "MARINA + DRIVE": DRIVE_METRICS
}

GAMMAS = {
    "DCGD + IQ": DCGD_UNCORR_GAMMA_MULT,
    # "DCGD + CQ": DCGD_CORR_GAMMA_MULT,
    "MARINA + CQ": CORR_GAMMA_MULT,
    "MARINA + IQ": UNCORR_GAMMA_MULT,
    "MARINA + DRIVE": DRIVE_GAMMA_MULT
}

fig = plot(weighted=False)
plt.show()

In [ ]:
fig.savefig("../images/baseline_comparison.pdf", bbox_inches='tight')

## 4.4 Weighted $\mathsf{MARINA}$

In [ ]:
SIZE = 1024
NUM_CLIENTS = 128

SEED = 0
RNG = np.random.default_rng(SEED)

In [ ]:
WEIGHTED_GAMMA_MULT = {0.0: 32, 0.5: 32, 10.0: 32, 100.0: 32}
UNWEIGHTED_GAMMA_MULT = {0.0: 2, 0.5: 2, 10.0: 2, 100.0: 2}
NOISES = [0.0, 10.0]

WEIGHTED_METRICS = {}
UNWEIGHTED_METRICS = {}

PROBLEM_PROPERTIES = {noise: {} for noise in NOISES}

In [ ]:
COMM_LIMIT = 1e8

for i, noise in enumerate(NOISES):
    print(f"noise = {noise}")
    functions = create_convex_different_liptschitz(NUM_CLIENTS, SIZE, RNG, noise_lambda=noise)
    
    liptschitz_constants = functions.get_liptschitz_constants()
    PROBLEM_PROPERTIES[noise]["minLi"] = min(liptschitz_constants)
    PROBLEM_PROPERTIES[noise]["maxLi"] = max(liptschitz_constants)
    
    functions.reset()
    UNWEIGHTED_METRICS[noise] = measure(
        functions, DriveQuantizer, UNWEIGHTED_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT, use_importance_sampling=False,
    )    
    functions.reset()
    WEIGHTED_METRICS[noise] = measure(
        functions, DriveQuantizer, WEIGHTED_GAMMA_MULT[noise], RNG, comm_limit=COMM_LIMIT, use_importance_sampling=True, batch_size=1,
    )


In [ ]:
METRICS = {
    "MARINA + DRIVE + IS": WEIGHTED_METRICS,
    "MARINA + DRIVE": UNWEIGHTED_METRICS,
}
GAMMAS = {
    "MARINA + DRIVE + IS": WEIGHTED_GAMMA_MULT,
    "MARINA + DRIVE": UNWEIGHTED_GAMMA_MULT
}

plt.rcParams["axes.formatter.limits"] = (5, 5)
fig = plot(weighted=True)
plt.show()

In [ ]:
fig.savefig("../images/weighted.pdf", bbox_inches='tight')

## B.3.3 Numerical Complexity Analysis in the d-n Plane

In [ ]:
from scipy.optimize import minimize
import numpy as np

In [ ]:
def get_func(a, b):
    def func_(p):
        return (p + (1 - p) * a) * (1 + np.sqrt((1 - p) / p * b))
    return func_

def get_grad(a, b):
    def grad_(p):
        return ((-b/p - b * (1-p)/p**2) * (p + a * (1-p)))/(2*np.sqrt(b * (1-p)/p)) + (1-a) * (1 + np.sqrt(b * (1 - p) / p))
    return grad_

In [ ]:
NUM=200
MAX = 14

ns = np.logspace(2, MAX, num=NUM)
ds = np.logspace(2, MAX, num=NUM)

In [ ]:
from tqdm import tqdm

def optimize_all(ns, ds, correlated: bool):
    optimum_values = np.empty((len(ns), len(ds)))
    
    prev=0.5
    for i, n in enumerate(tqdm(ns)):
        for j, d in enumerate(ds):
            if correlated:
                a, b = (32+d)/(32*d), 12*d/n**2
            else:
                a, b = (32+d)/(32*d), d/(4 * n)

            func = get_func(a, b)
            grad = get_grad(a, b)
            res = minimize(func, prev, method='SLSQP', jac=grad, bounds=[(1e-5, 1-1e-5)], tol=1e-10)
            assert res.success, res.message
            optimum_values[i, j] = min(func(res.x[0]), func(1))
            prev = res.x[0] if res.x[0] < 0.99 else 0.1
    return optimum_values

In [ ]:
correlated_values = optimize_all(ns, ds, correlated=True)
uncorrelated_values = optimize_all(ns, ds, correlated=False)
ratio = uncorrelated_values / correlated_values

In [ ]:
ns, ds = np.meshgrid(ns, ds)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}' #for \text command

fig, axs = plt.subplots(1, 3, figsize=(14, 5), gridspec_kw={'width_ratios': [1, 1, 1.25]})

ticks = range(10, NUM, NUM // MAX)
tick_labels = np.linspace(1, MAX, MAX)

ax = sns.heatmap(-np.log10(uncorrelated_values), ax=axs[0], cbar=False)
log_uncorrelated_values = np.log10(uncorrelated_values)
ax.set_xticks(ticks)
ax.set_xticklabels(tick_labels)
ax.set_xlabel(r'$\log_{10}{d}$' + '\n(a)')
ax.set_yticks(ticks)
ax.set_yticklabels(tick_labels)
ax.set_ylabel(r'$\log_{10}{n}$')
ax.set_title(r"$-\log_{10}{\text{IF}_{\text{ind}}}$")
n_opt, d_opt = np.argmin(log_uncorrelated_values) // NUM, np.argmin(log_uncorrelated_values) % NUM
ax.scatter([d_opt], [n_opt], label="Max Speedup\n=" + f"{1/uncorrelated_values[n_opt][d_opt]:.3}", color="green", marker="*")
ax.legend()

ax = sns.heatmap(-np.log10(correlated_values), ax=axs[1], cbar=False)
log_correlated_values = np.log10(correlated_values)
ax.set_xticks(ticks)
ax.set_xticklabels(tick_labels)
ax.set_xlabel(r'$\log_{10}{d}$' + '\n(b)')
ax.set_yticks([])
# ax.set_yticks(ticks)
# ax.set_yticklabels(tick_labels)
# ax.set_ylabel(r'$\log_{10}{n}$')
ax.set_title(r"$-\log_{10}{\text{IF}_{\text{cor}}}$")
n_opt, d_opt = np.argmin(log_correlated_values) // NUM, np.argmin(log_correlated_values) % NUM
ax.scatter([d_opt], [n_opt], label="Max Speedup\n=" + f"{1/correlated_values[n_opt][d_opt]:.3}", color="green", marker="*")
ax.legend()

ax = sns.heatmap(np.log10(ratio), ax=axs[2])
log_ration = np.log10(ratio)
ax.set_xticks(ticks)
ax.set_xticklabels(tick_labels)
ax.set_xlabel(r'$\log_{10}{d}$' + '\n(c)')
ax.set_yticks([])
# ax.set_yticks(ticks)
# ax.set_yticklabels(tick_labels)
# ax.set_ylabel(r'$\log_{10}{n}$')
ax.set_title(r"$\log_{10} \text{IF}_{\text{cor}} / \text{IF}_{\text{ind}}$")
n_opt, d_opt = np.argmax(log_ration) // NUM, np.argmax(log_ration) % NUM
ax.scatter([d_opt], [n_opt], label="Max Speedup\n=" + f"{ratio[n_opt][d_opt]:.3}", color="green", marker="*")
ax.legend()

# ax = axs[1][1]
# ax.plot(np.diagonal(ratio))
# ax.axhline(7.29, color="gray", linestyle="--")

fig.tight_layout()

plt.show()

In [ ]:
fig.savefig("../images/dn_plane.pdf", bbox_inches='tight')